# Project II: Economic Growth 

This notebook will help you getting started with analyzing the growth dataset, `growth.csv`.

In [1]:
import pandas as pd 
import numpy as np 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import estimation as est
from numpy import linalg as la

%load_ext autoreload
%autoreload 2
import tools as lm 

## Read data 
First, we need to read the data. The dataset is stored in the `growth.csv` file.

The data is then examined to understand its structure.

In [2]:
dat = pd.read_csv('data/ppcs_cc.csv')
# Add constant
dat['const'] = 1

# Check if the data is cross-sectional
assert len(dat.year.unique())==1

# Dimensions of the data
N, K = dat.shape

# Print the data
print(f'The data contains {N} rows and {K} columns (variables) and is cross-sectional \n')
print(f'Variables are \n {dat.columns.values} \n')
dat.head(5)

The data contains 3799 rows and 20 columns (variables) and is cross-sectional 

Variables are 
 ['sblack' 'shisp' 'swhite' 'sother' 'smale' 'sage' 'sempl' 'sincome'
 'spop' 'daytime' 'inctype_lin' 'omajblack' 'omajhisp' 'omajwhite'
 'omajother' 'osplit' 'sbehavior' 'year' 'anyuseofforce_coded' 'const'] 



,sblack,shisp,swhite,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajwhite,omajother,osplit,sbehavior,year,anyuseofforce_coded,const
0,1,0,0,0,1,18,0,1,1,1,2,0,0,1,0,0,0,2011,0,1
1,1,0,0,0,1,20,1,2,4,0,2,0,0,1,0,0,0,2011,0,1
2,1,0,0,0,1,22,1,2,3,1,2,0,0,1,0,0,0,2011,0,1
3,1,0,0,0,1,22,1,3,1,1,2,0,0,1,0,0,0,2011,0,1
4,1,0,0,0,1,22,1,1,1,1,2,0,0,1,0,0,0,2011,0,1


In [14]:
# create the array with the y-values
y = dat['anyuseofforce_coded'].values.reshape((N,))
y_lab = ['anyuseofforce_coded']

# list of characteristics 
race_list = ['sblack', 'shisp', 'swhite', 'sother']
s_char = ['smale','sage', 'sempl', 'sincome', 'spop', 'sbehavior']
o_char = ['omajblack', 'omajhisp', 'omajwhite','omajother', 'osplit'] 
other_char = ['daytime', 'inctype_lin']

# CONSIDER WHAT HAPPENS HERE
# s_char = ['sincome'] # removed to avoid multicollinearity
# o_char = ['omajwhite'] # removed to avoid multicollinearity
# other_char = ['daytime'] # removed to avoid multicollinearity


##

In [11]:

print('Average use of force for black/white (if higher then indication of police force) \n', 
      dat.groupby('sblack')['anyuseofforce_coded'].mean(), 
      '\n')

# Create a list of race variables and run regression
x_lab = ['const','sblack', 'shisp', 'sother']
x = dat[x_lab].values.reshape((N,len(x_lab)))

# Run regression by getting starting values and then estimating the model
theta0 = lm.starting_values(y,x)
ll = lm.loglikelihood(theta0, y, x); #print(f"Using the starting values {theta0}, the log sum of likelihood is {ll.sum():,.0f}")
logit_results = est.estimate(lm.q, theta0, y, x, options={'disp': False})
# Print the results
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab[0]}', )
logit_tab

Average use of force for black/white (if higher then indication of police force) 
 sblack
0    0.004735
1    0.007143
Name: anyuseofforce_coded, dtype: float64 

Optimizer succeded after 51 iter. (265 func. evals.). Final criterion:  0.03044.
Logit, y = anyuseofforce_coded


,theta,se,t
const,-5.7413,0.3339,-17.1963
sblack,0.8056,0.6687,1.2046
shisp,1.5934,0.5299,3.0072
sother,0.5345,1.0568,0.5058


In [15]:
def create_var_list(coef_interest, sets:list):
    """ List of variables to include in the regression."""
    # sets = [s_char, o_char, other_char]
    x_labels = [['const', coef_interest]]  # Start with just the constant

    # Add all variables in stages
    for i, group in enumerate(sets):
        i += 1
        x_labels.append(x_labels[i-1] + group)
    
    x_values = []
    for x_list in x_labels:
        x = dat[x_list].values.reshape((N,len(x_list)))
        x_values.append(x)
    
    return x_values, x_labels

## Example of probit and logit Estimation and average partial effects 

This illustrates how to estimate a probit and logit model and calculate average partial effects for the model.

$\mathcal{I} \in (\text{Income}, \text{Male}, \text{Age}, \text{Employment})$

In [43]:
coef_interest = 'sblack'

# Create the list of variables
xLabels = ['const',coef_interest,'smale','sage','sincome', 'sempl']
xValues = dat[xLabels].values.reshape((N,len(xLabels)))

# test if the matrix is full rank
if la.matrix_rank(xValues) < xValues.shape[1]:
    raise ValueError('The matrix is not full rank')

# estimate the model logit model
logit_results = est.estimate(lm.q, theta0=lm.starting_values(y, xValues), y=y, x=xValues, options={'disp': False})

for i in range(len(xLabels)):
    print(f"Estimation of {xLabels[i]}: {logit_results['theta'][i]:10.3f} with std: {np.sqrt(logit_results['cov'][i,i]):10.3f}")
print()

# calculate the average marginal effects
ape, ape_var, pe_sample_std = lm.average_partial_effect(x_i=xValues,                      # x-values
                                                         betas=logit_results['theta'],    # estimated coefficients
                                                         cov_matrix=logit_results['cov'], # covariance matrix
                                                         k=1 # location in the xLabels of the variable of interest
                                                         )

print(f"Average Partial Effect of {coef_interest}: {ape:4.3f} with std: {np.sqrt(ape_var[0])[0]:4.3f}")


Estimation of const:     -3.945 with std:      1.082
Estimation of sblack:      0.341 with std:      0.662
Estimation of smale:      1.298 with std:      0.673
Estimation of sage:     -0.044 with std:      0.025
Estimation of sincome:     -0.005 with std:      0.284
Estimation of sempl:     -1.117 with std:      0.522

Average Partial Effect of sblack: 0.002 with std: 0.003


### Creating a loop to estimate the model for each variable

In [89]:
coef_interest = 'sblack'

race_list = ['sblack', 'shisp', 'swhite', 'sother']

officer_char = ['omajblack', 'omajwhite','omajother'] # removed: 'omajother'
other_char = ['daytime']

resDict = {}
for i, coef_interest in enumerate(race_list):
    print(f'Estimating the model with {coef_interest} as the variable of interest')
    # initialize the result dictionary to store the results
    resDict[coef_interest] = {}

    # Create the list of variables
    xLabels = ['const',coef_interest,'smale','sage','sincome', 'sempl']
    xValues = dat[xLabels].values.reshape((N,len(xLabels)))

    # loop over the characteristics
    for addChar in [[], officer_char, other_char]: # Adding an empty to run standard model
        # add the characteristics
        xLabels = xLabels + addChar
        xValues = dat[xLabels].values.reshape((N,len(xLabels)))

        print(f'Estimating the model with {xLabels} as the variables')
                                              
        # test if the matrix is full rank
        if la.matrix_rank(xValues) < xValues.shape[1]:
            print('The matrix is not full rank')
            print(xValues)
            continue

        # estimate the model logit model
        logit_results = est.estimate(lm.q, theta0=lm.starting_values(y, xValues), y=y, x=xValues, options={'disp': False})

        # store the results
        resDict[coef_interest] = {'xLabels': xLabels, 'logit_theta': logit_results['theta'], 'logit_cov': np.diag(logit_results['cov'])}
        
        # calculate the average marginal effects
        ape, ape_var, pe_sample_std = lm.average_partial_effect(x_i=xValues,                      # x-values
                                                                betas=logit_results['theta'],    # estimated coefficients
                                                                cov_matrix=logit_results['cov'], # covariance matrix
                                                                k=1 # location in the xLabels of the variable of interest
                                                                )
        
        # Add ape an ape_var to the dictionary
        resDict[coef_interest]['ape'] = ape
        resDict[coef_interest]['ape_var'] = ape_var[0][0]
    

Estimating the model with sblack as the variable of interest
Estimating the model with ['const', 'sblack', 'smale', 'sage', 'sincome', 'sempl'] as the variables
Estimating the model with ['const', 'sblack', 'smale', 'sage', 'sincome', 'sempl', 'omajblack', 'omajwhite', 'omajother'] as the variables
Estimating the model with ['const', 'sblack', 'smale', 'sage', 'sincome', 'sempl', 'omajblack', 'omajwhite', 'omajother', 'daytime'] as the variables
Estimating the model with shisp as the variable of interest
Estimating the model with ['const', 'shisp', 'smale', 'sage', 'sincome', 'sempl'] as the variables
Estimating the model with ['const', 'shisp', 'smale', 'sage', 'sincome', 'sempl', 'omajblack', 'omajwhite', 'omajother'] as the variables
Estimating the model with ['const', 'shisp', 'smale', 'sage', 'sincome', 'sempl', 'omajblack', 'omajwhite', 'omajother', 'daytime'] as the variables
Estimating the model with swhite as the variable of interest
Estimating the model with ['const', 'swhite

In [87]:
df_logit = pd.DataFrame.from_dict(resDict, orient='index').drop(columns=['ape', 'ape_var'])
# expand the list of coefficients
(pd.concat([df_logit['xLabels'].apply(pd.Series).T.stack(),
           df_logit['logit_theta'].apply(pd.Series).T.stack(), 
           df_logit['logit_cov'].apply(pd.Series).T.stack()
           ], axis=1)
           .rename(columns={0: 'label', 1: 'coeff',2:'var'})
           .reset_index(drop=False)
           .rename(columns={'level_1': 'sRace'})
           .drop(columns=['level_0'])
           .set_index(['sRace','label'])
           .sort_values(by='sRace')
           )

coeff           var
sRace  label                             
sblack const      -2.982453  2.107942e+00
       sempl      -1.072456  3.145033e-01
       sincome     0.007400  8.272455e-02
       omajwhite  -0.738053  1.728815e+00
       sage       -0.041421  6.312939e-04
       omajother  -4.946166  3.322738e+06
       smale       1.258282  5.439780e-01
       omajblack  -8.359403  1.992901e+08
       sblack      0.471010  5.952509e-01
       daytime    -0.549099  3.897451e-01
shisp  smale       1.198153  5.419361e-01
       shisp       1.091980  3.550674e-01
       sempl      -1.045155  2.857279e-01
       sage       -0.037726  6.269968e-04
       omajwhite  -0.380817  1.927898e+00
       daytime    -0.574001  3.214374e-01
       sincome     0.041640  8.946795e-02
       omajblack  -7.929155  3.135805e+08
       const      -3.638947  3.084384e+00
       omajother  -5.968413  2.793198e+07
sother omajother  -6.774720  1.483804e+08
       omajblack -11.063728  7.160943e+10
       omajwhite  -0.742815  1.637651e+00
       sincome    -0.015392  1.020186e-01
       sage       -0.041193  1.173084e-03
       smale       1.253655  5.393766e-01
       sother     -0.194166  2.509074e+00
       const      -2.854505  2.469528e+00
       sempl      -1.078820  2.817698e-01
       daytime    -0.558731  3.415586e-01
swhite sempl      -1.039612  2.829289e-01
       daytime    -0.512397  2.861722e-01
       omajblack -10.577328  2.226074e+10
       sincome     0.062871  8.272216e-02
       sage       -0.038635  6.224488e-04
       omajwhite  -0.523257  1.707592e+00
       smale       1.183566  5.478256e-01
       omajother  -7.244651  2.020292e+08
       swhite     -0.952677  2.365750e-01
       const      -2.741891  2.033440e+00

In [88]:
pd.DataFrame.from_dict(resDict, orient='index')[['ape', 'ape_var']]

,ape,ape_var
sblack,0.002787,0.000017
shisp,0.007320,0.000036
swhite,-0.006139,0.000034
sother,-0.000868,0.000059
